## Data-preparatie

Dit notebook behandelt de data-preparatie en geeft antwoord op deelvraag 1:

*"Hoe kan cellijn-data (mutaties, methylatie, drug-respons) worden verzameld, opgeschoond en geschikt gemaakt voor machine learning?"*

Voor dit onderzoek worden vier afzonderlijke datasets gebruikt die afkomstig zijn van het CCLE-project (Cancer Cell Line Encyclopedia). De data beschrijven verschillende moleculaire kenmerken en annotaties van kankercellijnen:

- OmicsSomaticMutations.csv - bevat de somatische mutatieprofielen van de cellijnen.
- CCLE_RRBS_TSS1kb_20181022.txt - bevat DNA-methylatiegegevens gemeten rond de Transcription Start Site (TSS, ±1 kb). Dit gebied omvat zowel promoterregio's als het eerste exon, die belangrijk zijn voor genregulatie.
- Drug_sensitivity_AUC_(PRISM_Repurposing_Secondary_Screen)_subsetted.csv - bevat therapieresponsgegevens (AUC-waarden) die aangeven hoe gevoelig een cellijn is voor een bepaald medicijn.
- Cell_lines_annotations_20181226.txt - metadata-bestand met annotaties voor elke cellijn, inclusief `depMapID`, weefseltype (Site_Primary), histologie en andere klinisch-biologische kenmerken. Dit bestand wordt gebruikt om cellijnen te filteren op weefseltype en om gelijkmatigheid te creëren tussen de verschillende datasets. 

Het uiteindelijke doel is om een dataset te creëren waarin elke longkankercellijn zowel moleculaire kenmerken (mutaties en methylatie) als een responswaarde (AUC) bevat, zodat deze kan worden gebruikt als input voor supervised machine learning.

**xxx** toelichting CpG sites en avg coverage (kwaliteitsmaat) xxx:
CpG_sites_hg19 = genomische voordinaten van CpG-sites in het Hg19-genoom
avg_coverage = maat voor de betrouwbaarheid van de methylatie-metingen op dat locus

TSS=Transcription Start Site, het beginpunt van een gen waar transcriptie start.
±1kb betekent dat de gemeten regio 1 kilobase (1000bp) upstream en downstream van de TSS ligt
dus direct rond het begin van het gen, inclusief een stukje voor (promotorgebied) en erna (eerste exon/regio vlakbij TSS)

- Upstream van TSS (~1000 bp): dit is meestal de promoter. Methylatie hier kan de transcriptie van het gen onderdrukken.
- Downstream van TSS (~1000 bp): dit is vaak het 5’ einde van het gen, inclusief exon 1 of de 5’ UTR. Methylatie hier kan ook effect hebben op genexpressie. methylatie in deze regio kan het initiatiefase van transcriptie veranderen, waardoor het gen minder of anders wordt afgelezen

Nadenken over avg_coverage. minimum instellen. te lagen waarden weglaten omdat de methylatiepercentages dan ruis bevatten (in de literatuur 10>)
laat ook zien want de impact is van die drempel.


#### Stap 1. Data verkennen

Doel: inzicht krijgen in de structuur van elk bestand voor het filteren en samenvoegen.

Het is belangrijk om de kolomnamen, rijenstructuur en datatypes te begrijpen. Zo krijg je inzicht in welke kolommen relevant zijn (bijv. cell_line_id, gene, AUC, avg_coverage) en kun je bepalen hoe de datasets aan elkaar gekoppeld kunnen worden.

stappen:
Controleer het aantal rijen/kolommen.
Bekijk een paar voorbeeldrijen om de structuur te snappen.
Noteer de naam van de kolom die de cellijn identificeert (ID).
Kijk of de kolomnamen consistent zijn tussen bestanden.
Kijk hoeveel unieke cellijnen er per bestand zijn.))


In [56]:
import pandas as pd

# aanmaken variabele met de bestandpad
mutatie_file = "../data/raw/OmicsSomaticMutations.csv"

# inlezen 
mutatie_df = pd.read_csv(mutatie_file, low_memory=False) # csv, geen tabs

# verkenning
mutatie_df.info()
print(mutatie_df.head()) 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 751310 entries, 0 to 751309
Data columns (total 70 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Chrom                            751310 non-null  object 
 1   Pos                              751310 non-null  int64  
 2   Ref                              751310 non-null  object 
 3   Alt                              751310 non-null  object 
 4   AF                               751310 non-null  float64
 5   DP                               751310 non-null  int64  
 6   RefCount                         751310 non-null  int64  
 7   AltCount                         751310 non-null  int64  
 8   GT                               751310 non-null  object 
 9   PS                               44923 non-null   float64
 10  VariantType                      751310 non-null  object 
 11  VariantInfo                      751310 non-null  object 
 12  DN

In [119]:
mutatie_df['ModelID'].nunique()

1939

De mutatie-dataset bevat 751310 rijen en 70 kolommen en bestaat uit de volgende datatypen:

- object: tekst/categorieën (bijvoorbeeld genen en mutatietypes)
- float64: decimale waarden
- bool: waar/onwaar waarden (True/False)
- int64: hele getallen (bijvoorbeeld aantallen mutaties)

Belangrijk voor dit onderzoek:
ModelID: koppeling tussen datasets
HugoSymbol/EnsemblGeneID: gen waarin mutatie voorkomt
VariantType: type mutatie (feature voor ML)
DNAChange/ProteinChange: optioneel voor interpretatie
AF, RefCount, AltCount: betrouwbaarheid / filteren
LikelyLoF, OncogeneHighImpact, TumorSuppressorHighImpact: impact op functie/belangrijke features
Sift, Polyphen, RevelScore: schadelijkheidsvoorspelling
PrimaryDisease/Tissue: filteren op longkankercellijnen

In [58]:
# aanmaken variabele met de bestandpad
methylatie_file = "../data/raw/CCLE_RRBS_TSS1kb_20181022.txt"

# inlezen
methylatie_df = pd.read_csv(methylatie_file, sep='\t', low_memory=False) # txt, tab seperated

# verkenning
print("Aantal rijen en kolommen:", methylatie_df.shape) # globale info

# eerste en laatste 5 kolmmen bekijken
print("Eerste 5 kolommen:", methylatie_df.columns[:10].tolist())
print("Laatste 5 kolommen:", methylatie_df.columns[-10:].tolist())

# datatypes en missende waarden per kolom bekijken
kolom_info_methylatie = pd.DataFrame({ # aanmaken dataframe (met 3 kolommen)
    'kolom': methylatie_df.columns,
    'dtype': [methylatie_df[col].dtype for col in methylatie_df.columns],
    'n_missing': [methylatie_df[col].isna().sum() for col in methylatie_df.columns] # bepaald totaal aantal missende waarden voor elke kolom
})

# eerste 10 en laatste 10 kolommen tonen als voorbeeld
print(kolom_info_methylatie.head(10)) # eerste 10 kolommen
print(kolom_info_methylatie.tail(10)) # laatste 10 kolommen

Aantal rijen en kolommen: (21338, 846)
Eerste 5 kolommen: ['locus_id', 'CpG_sites_hg19', 'avg_coverage', 'DMS53_LUNG', 'SW1116_LARGE_INTESTINE', 'P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE', 'HUT78_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE', 'UMUC3_URINARY_TRACT', 'HOS_BONE', 'AML193_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE']
Laatste 5 kolommen: ['JHUEM7_ENDOMETRIUM', 'OE21_OESOPHAGUS', 'MOLT3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE', 'HOP62_LUNG', 'EKVX_LUNG', 'UO31_KIDNEY', 'SF268_CENTRAL_NERVOUS_SYSTEM', 'SF539_CENTRAL_NERVOUS_SYSTEM', 'SNB75_CENTRAL_NERVOUS_SYSTEM', 'HOP92_LUNG']
                                       kolom   dtype  n_missing
0                                   locus_id  object          0
1                             CpG_sites_hg19  object          1
2                               avg_coverage  object          0
3                                 DMS53_LUNG  object          0
4                     SW1116_LARGE_INTESTINE  object          0
5   P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE 

De methylatie-dataset bevat 21338 rijen en 846 kolommen. Elke rij vertegenwoordigt een specifieke CpG-locus, een locatie op het DNA waar methylatie kan optreden.

De eerste kolommen bevatten metadata:
- `locus_id`: unieke identifier van de CpG-locus
- `CpG_sites_hg19`:genoompositie van de locus
- `avg_coverage`: gemiddelde meetdiepte, een maat voor de betrouwbaarheid van de meting

Alle overige kolommen (vanaf kolom 4) vertegenwoordigen de cellijnen. Wat me opvalt is dat dit bestand geen ACH- of ModelID bevat. In plaats daarvan worden cellijnen geidentificeerd op basis van hun naam,  waarbij de kolomtitel zowel de cellijnnam als het weefseltype bevat  (bijvoorbeeld `DMS53_LUNG`).

**Let op!** Om de drie datasets samen te voegen zal een mapping moeten worden gemaakt van deze kolomnamen naar de bijbehorende ACH_ID. Dit zal worden uitgevoerd in stap 5 (samenvoegen). 

In [46]:
# aanmaken variabele met de bestandpad
response_file = "../data/raw/Drug_sensitivity_AUC_(PRISM_Repurposing_Secondary_Screen)_subsetted.csv"
# inlezen
response_df = pd.read_csv(response_file, sep=',') # comma seperated

# verkenning
print("Aantal rijen en kolommen:", response_df.shape) # geeft het aantal rijen en kolommen

# datatypes en missende waarden per kolom bekijken
kolom_info_response = pd.DataFrame({
    'kolom': response_df.columns,
    'dtype': [response_df[col].dtype for col in response_df.columns],
    'n_missing': [response_df[col].isna().sum() for col in response_df.columns]
})

# eerste 20 kolommen tonen als voorbeeld
print(kolom_info_response.head(10))
print(kolom_info_response.tail(10))

print(response_df.iloc[:5,:10])

Aantal rijen en kolommen: (480, 1457)
                                               kolom    dtype  n_missing
0                                          depmap_id   object          0
1                             cell_line_display_name   object          0
2                                          lineage_1   object          0
3                                          lineage_2   object          0
4                                          lineage_3   object          0
5                                          lineage_6   object        436
6                                          lineage_4  float64        480
7          8-BROMO-CGMP (BRD:BRD-A00077618-236-07-6)  float64        370
8          NORETYNODREL (BRD:BRD-A00758722-001-04-9)  float64         98
9  PREDNISOLONE-ACETATE (BRD:BRD-A01643550-001-04-9)  float64        432
                                                  kolom    dtype  n_missing
1447            LORLATINIB (BRD:BRD-K99879819-001-02-1)  float64        326
1448   

De response-dataset bevat 480 rijen, waarbij elke rij een unieke cellijn representeert, en 1457 kolommen. 
De eerste kolommen bevatten meta-data over de cellijnen:

- depmap_id: unieke ID voor elke cellijn (bijvoorbeeld ACH-000973)
- cell_line_display_name: leesbare naam van de cellijn (bijvoorbeeld 639V of BECKER)
- lineage_1 t/m lineage_6: beschrijving van de biologische afkomst van de cellijn, van breed tot specifiek.

Vanaf kolom 8 bevatten de overige kolommen de drug-response waarden voor verschillende geneesmiddelen, weergegeven als numerieke scores (AUC). Elke cellijn is getest voor meerdere geneesmiddelen, waarbij sommige waarden ontbreken (NaN).


In [129]:
# inlezen van het metadata-bestand
metadata_file = "../data/raw/Cell_lines_annotations_20181226.txt"
metadata_df = pd.read_csv(metadata_file, sep='\t', low_memory=False) # txt, tab seperated

# verkenning
print("Aantal rijen en kolommen:", metadata_df.shape) # globale info

print(metadata_df.columns)

# belangrijke kolommen selecteren
cols = ['CCLE_ID', 'depMapID', 'Name', 'Site_Primary', 
        'Site_Subtype1', 'Site_Subtype2', 'Site_Subtype3', 'Disease']

# alleen de eerste rijen met deze kolommen tonen
print(metadata_df[cols].head())


Aantal rijen en kolommen: (1461, 33)
Index(['CCLE_ID', 'depMapID', 'Name', 'Pathology', 'Site_Primary',
       'Site_Subtype1', 'Site_Subtype2', 'Site_Subtype3', 'Histology',
       'Hist_Subtype1', 'Hist_Subtype2', 'Hist_Subtype3', 'Gender',
       'Life_Stage', 'Age', 'Race', 'Geo_Loc', 'inferred_ethnicity',
       'Site_Of_Finding', 'Disease', 'Annotation_Source',
       'Original.Source.of.Cell.Line', 'Characteristics', 'Growth.Medium',
       'Supplements', 'Freezing.Medium', 'Doubling.Time.from.Vendor',
       'Doubling.Time.Calculated.hrs', 'type', 'type_refined',
       'PATHOLOGIST_ANNOTATION', 'mutRate', 'tcga_code'],
      dtype='object')
                                    CCLE_ID    depMapID       Name  \
0                                DMS53_LUNG  ACH-000698     DMS 53   
1                    SW1116_LARGE_INTESTINE  ACH-000489     SW1116   
2                             NCIH1694_LUNG  ACH-000431  NCI-H1694   
3  P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE  ACH-000707     P3

beschrijving output metadata
aantal rijen en kolomme, de kolommen en de eerste 5 regels van de relevante kolommen die nodig zijn ...

#### Stap 2. Filteren op relevante data

Doel: alleen de biologische en analyse relevante mutaties behouden, zodat de dataset overzichtelijk blijft en rekentijd wordt bespaard.

**Mutatiebestand**

Acties:
1. filteren op cellijntype: alleen cellijnen van het weefseltype "lung" worden behouden.
2. filteren op relevante mutaties: niet alle mutaties zijn biologisch significant (veel varianten zijn neutraal of technisch artefact). Daarom worden alleen functionele mutaties behouden op basis van de volgende kolommen:
   - LiklyLoF (verlies-van-functie mutaties)
   - OncogeneHighImpact (activerende mutaties in oncogenen)
   - TumorSuppressorHighImpact (verlies-van-remfunctie in tumorsuppressorgen)
   - Hotspot (bekende activerende hotspot-mutaties)
   - HessDriver (mutaties geidentificeerd als drivers)
   Een mutatie wordt behouden als één of meer van deze criteria waar zijn.
3. transformatie naar binaire matrix: voor elke cellijn (ModelID) wordt per gen (HugoSymbol) gekeken of er een relevante mutate aanwezig is. Wanneer een mutatie wordt gevonden dan krijgt deze waarde=1, anders waarde=0. Dit resulteert in een binaire matrix waarin rijen en cellijnen representeren en kolommen genen.


In [123]:
# kolommen waar long kan voorkomen
tissue_cols = ['Site_Primary', 'Site_Subtype1', 'Site_Subtype2', 'Site_Subtype3']

# filteren op long 
lung_meta = metadata_df[metadata_df[tissue_cols].apply(lambda x: x.str.contains('lung', case = False, na = False)).any(axis=1)
]

# depMapIDs van longkankercellijnen ophalen
lung_depmap_ids = lung_meta['depMapID'].tolist()

# filter mutatie_df op longkankercellijnen
lung_mutatie_df = mutatie_df[mutatie_df['ModelID'].isin(lung_depmap_ids)]

# check resultaat
print(lung_mutatie_df.head())
print(f"Aantal mutaties in longkankercellijnen in mutatie_df: {lung_mutatie_df.shape[0]}")

# unieke ModelIDs in gefilterde Dataframe op lung
aantal_lung_cellijnen = lung_mutatie_df['ModelID'].nunique()
print(f"Aantal longkankercellijnen: {aantal_lung_cellijnen}")



   Chrom     Pos Ref Alt     AF  DP  RefCount  AltCount   GT  PS  ...  \
0   chr1  818203   G   A  0.240  27        21         6  0/1 NaN  ...   
15  chr1  939091   G   A  0.500  30        15        15  0/1 NaN  ...   
17  chr1  939389   A   C  0.250  10         8         2  0/1 NaN  ...   
26  chr1  942868   A   C  0.649  46        16        30  0/1 NaN  ...   
39  chr1  944763   C  CA  0.247   9         7         2  0/1 NaN  ...   

   GwasPmID GtexGene ProveanPrediction AMClass AMPathogenicity Rescue  \
0       NaN      NaN               NaN     NaN             NaN  False   
15      NaN      NaN          Damaging     NaN             NaN  False   
17      NaN      NaN          Damaging     NaN             NaN  False   
26      NaN      NaN           Neutral     NaN             NaN  False   
39      NaN      NaN               NaN     NaN             NaN  False   

   RescueReason     ModelID Hotspot EntrezGeneID  
0           NaN  ACH-000062   False     400728.0  
15          NaN  ACH

In [124]:
# filter alleen biologisch relevante mutaties
filtered = lung_mutatie_df[
    lung_mutatie_df['LikelyLoF'] |
    lung_mutatie_df['OncogeneHighImpact'] |
    lung_mutatie_df['TumorSuppressorHighImpact'] |
    lung_mutatie_df['Hotspot'] |
    lung_mutatie_df['HessDriver']
]

# Maak binaire matrix per gen per celijn
mut_matrix = (
    filtered
    .assign(mutated=1)
    .pivot_table(index='ModelID', columns='HugoSymbol', values='mutated', fill_value=0)
    .astype(int) 
)

print(mut_matrix.shape)
print(mut_matrix.iloc[:,:5].head)

(174, 7358)
<bound method NDFrame.head of HugoSymbol  A1CF  A2M  A2ML1  A4GALT  AAAS
ModelID                                   
ACH-000012     0    0      0       0     0
ACH-000015     0    0      0       0     0
ACH-000021     0    0      0       0     0
ACH-000029     0    0      0       0     0
ACH-000030     0    0      0       0     0
...          ...  ...    ...     ...   ...
ACH-001049     0    0      0       0     0
ACH-001075     0    0      0       0     0
ACH-001113     0    0      0       0     0
ACH-001138     0    0      0       0     0
ACH-001233     0    0      0       0     0

[174 rows x 5 columns]>


Output: 
- de matrix bevat 174 rijen (één per longkankercellijn) en 7358 kolommen (één per gen).
- elke rij komt overeen met een unieke cellijn (ModelID) die in de metadata geclassificeerd is als afkomstig van longweefsel.
- elke kolom stelt een gen voor (`HugoSymbol`) dat ten minste één van de longkankercellijnen een relevante mutatie bevat.
- de celwaarden zijn binair:
    - 1 = het gen bevat minstens één relevante mutatie in die cellijn.
    - 0 = het gen bevat geen relevante mutatie in die cellijn.


In [ ]:
Methylatie bestand filteren 
xxx


In [100]:
# check kolomnamen
print(methylatie_df.columns[:20])

Index(['locus_id', 'CpG_sites_hg19', 'avg_coverage', 'DMS53_LUNG',
       'SW1116_LARGE_INTESTINE', 'P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE',
       'HUT78_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE', 'UMUC3_URINARY_TRACT',
       'HOS_BONE', 'AML193_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE', 'RVH421_SKIN',
       'NCIH1184_LUNG', 'HCC2157_BREAST', 'NCIH2227_LUNG', 'SNU449_LIVER',
       'NCIH28_PLEURA', 'OV56_OVARY', 'RMUGS_OVARY',
       'LN229_CENTRAL_NERVOUS_SYSTEM',
       'P31FUJ_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE'],
      dtype='object')


Probleem: Methylatiebestand heeft geen DepMap- of ACH-ID's, maar gebrubikt kolomnamen die bestaan uit cellijnnaam + weefseltype (zoals DMS53_LUNG, SW1116_LARGE_INTESTINE). Elke kolom is een cellijn en elke rij is een metylatie-feature (CpG-site).

In plaats van filteren op rijen, moeten de kolommen worden geselecteerd die bij longkanker horen (dus de kolommen waarvan de naam "_LUNG" bevat.


In [125]:
# filter kolommen waar "LUNG" in voorkomt
lung_columns = [col for col in methylatie_df.columns if 'lung' in col.lower()]

# maak dataframe met longkankercellijnen + locusinfo
lung_methylatie_df = methylatie_df[['locus_id', 'CpG_sites_hg19', 'avg_coverage'] + lung_columns]

print(f"Aantal longkankerkolommen: {len(lung_columns)}")
print(lung_methylatie_df.shape)
print(lung_methylatie_df.head)
      


Aantal longkankerkolommen: 153
(21338, 156)
<bound method NDFrame.head of                              locus_id  \
0           SGIP1_1_66998638_66999638   
1           SGIP1_1_66998251_66999251   
2           AZIN2_1_33545713_33546713   
3           AZIN2_1_33546778_33547778   
4           AGBL4_1_50489626_50490626   
...                               ...   
21333      SYCE3_22_51001381_51002381   
21334     RABL2B_22_51222110_51223110   
21335  RPL23AP82_22_51194513_51195513   
21336  RPL23AP82_22_51221156_51222156   
21337                           NA_NA   

                                          CpG_sites_hg19 avg_coverage  \
0      1:66998970;1:66998973;1:66998993;1:66999404;1:...        25.00   
1                       1:66998970;1:66998973;1:66998993         8.27   
2      1:33546151;1:33546209;1:33546210;1:33546385;1:...       326.58   
3      1:33546783;1:33546788;1:33546795;1:33546797;1:...       480.54   
4      1:50489632;1:50489641;1:50489671;1:50489677;1:...       263.3

uitleg over dat aantal features te hoog is, hoe verder filteren

filteren op kwaliteit (avg coverage)


In [111]:
# maak kopie 
lung_methylatie_df = lung_methylatie_df.copy()

# converteer avg_coverage naar numeriek
lung_methylatie_df['avg_coverage'] = pd.to_numeric(lung_methylatie_df['avg_coverage'], errors ='coerce')

#verwijder rijen met NaN in avg_coverage
lung_methylatie_df = lung_methylatie_df.dropna(subset=['avg_coverage'])

# filter loci met avg_coverage >=10
filtered_10_df = lung_methylatie_df[lung_methylatie_df['avg_coverage']>=10]

# check hoe veel loci overblijven
print(f"Loci voor filtering op avg coverage: {len(lung_methylatie_df)}")
print(f"Loci na filtering op avg coverage: {len(filtered_10_df)}")


Loci voor filtering op avg coverage: 21337
Loci na filtering op avg coverage: 20898


uitleg: voor ML zijn dit nog steeds te veel features (zeker voor 100-200 cellijnen).
nu variantie-filtering:
Bij DNA-methylatiegegevens worden duizende CpG-loci gemeten. Veel van deze GpC loci variëren nauwelijks tussen cellijnen. Deze lage-variantie loci voegen weinig voorspellende waarde toe voor machine learning en verhogen alleen de ruis en het aantal features. 


In [116]:
import numpy

# Maak een kopie zodat we veilig bewerkingen kunnen doen
filtered_10_df = filtered_10_df.copy()

# Lijst van kolommen met alleen longkankercellijn-methylatie
methylation_cols = [col for col in filtered_10_df.columns if col not in ['locus_id', 'CpG_sites_hg19', 'avg_coverage']]

# Converteer alle waarden naar numeriek, foutieve waarden worden NaN
filtered_10_df[methylation_cols] = filtered_10_df[methylation_cols].apply(pd.to_numeric, errors='coerce')

# Optioneel: verwijder loci waar ALLE cellijnen NaN zijn
filtered_10_df = filtered_10_df.dropna(subset=methylation_cols, how='all')

# Bereken de standaardafwijking per locus over alle longkankercellijnen
std_per_locus = filtered_10_df[methylation_cols].std(axis=1)

# Stel drempel in: loci met std >= 0.05 behouden
threshold = 0.05
filtered_var_df = filtered_10_df[std_per_locus >= threshold]

print(f"Aantal loci voor variantie-filtering: {len(filtered_10_df)}")
print(f"Aantal loci na variantie-filtering: {len(filtered_var_df)}")


Aantal loci voor variantie-filtering: 20898
Aantal loci na variantie-filtering: 12434


In [117]:
# Bekijk de eerste 5 rijen en 10 kolommen van de dataset
print(filtered_var_df.iloc[:5, :10])
# Bekijk de eerste 5 rijen en alle methylatiekolommen
methylation_cols = [col for col in filtered_var_df.columns if col not in ['locus_id', 'CpG_sites_hg19', 'avg_coverage']]
print(filtered_var_df[methylation_cols].head())


                    locus_id  \
0  SGIP1_1_66998638_66999638   
2  AZIN2_1_33545713_33546713   
3  AZIN2_1_33546778_33547778   
4  AGBL4_1_50489626_50490626   
6  CLIC4_1_25070759_25071759   

                                      CpG_sites_hg19  avg_coverage  \
0  1:66998970;1:66998973;1:66998993;1:66999404;1:...         25.00   
2  1:33546151;1:33546209;1:33546210;1:33546385;1:...        326.58   
3  1:33546783;1:33546788;1:33546795;1:33546797;1:...        480.54   
4  1:50489632;1:50489641;1:50489671;1:50489677;1:...        263.36   
6  1:25070902;1:25070906;1:25071609;1:25071616;1:...        304.38   

   DMS53_LUNG  NCIH1184_LUNG  NCIH2227_LUNG  RERFLCAD2_LUNG  NCIH2347_LUNG  \
0     0.00000        0.11864        0.04545             NaN        0.25000   
2     0.00729        0.22553        0.04560         0.14577        0.11487   
3     0.22276        0.10735        0.03326         0.09589        0.21004   
4     0.00000        0.07346        0.01246         0.00531        0.07422

Mapping en transponeren van methylatie DataFrame

In deze stap voeren we twee acties uit:
1. mapping van kolommen naar ACH-IDs
2. Transponseren van de DataFrame

Hierbij wordt ook een snelle controle uitgevoerd om te zien hoeveel cellijnen in methylatie en mutatie voorkomen en hoeveel in beide datasets overlappen. Dit is een check waarbij nog geen cellijnen worden verwijderd, omdat er later nog responswaarden bij komen

In [133]:
# Actie 1. mapping
# alleen de methylatiekolommen van de overgebleven loci
methylation_cols = [col for col in filtered_var_df.columns 
                    if col not in ['locus_id','CpG_sites_hg19','avg_coverage']]

# maak mapping: CCLE_ID (kolomnaam) -> depMapID
ccle2depmap = dict(zip(metadata_df['CCLE_ID'], metadata_df['depMapID']))

# map kolommen en filter de cellijnen die geen mapping hebben eruit
new_cols = []
valid_cols = []
for col in methylation_cols:
    depmap_id = ccle2depmap.get(col)
    if depmap_id:  # alleen keepen als mapping bestaat
        new_cols.append(depmap_id)
        valid_cols.append(col)

# pas mapping toe op DataFrame
meth_mapped_df = filtered_var_df[['locus_id','CpG_sites_hg19','avg_coverage'] + valid_cols].copy()
meth_mapped_df.columns = ['locus_id','CpG_sites_hg19','avg_coverage'] + new_cols

print(f"Aantal cellijnen na mapping: {len(new_cols)}")
print(meth_mapped_df.head())

# Actie 2. transponeren 
# transponeer methylatie zodat cellijnen rijen zijn en loci kolommen
meth_T = meth_mapped_df.drop(columns=['locus_id','CpG_sites_hg19','avg_coverage']).T
meth_T.index.name = 'ModelID'  # ACH-ID als index, net als in mut_matrix
# Zorg dat de index in beide datasets de ACH-ID is
meth_ids = set(meth_T.index)        # methylatie cellijnen
mut_ids = set(mut_matrix.index)     # mutatie cellijnen

# aantal unieke cellijnen in elk bestand
print(f"Aantal cellijnen in methylatie: {len(meth_ids)}")
print(f"Aantal cellijnen in mutaties: {len(mut_ids)}")

# aantal cellijnen die in beide bestanden voorkomen
common_ids = meth_ids.intersection(mut_ids)
print(f"Aantal cellijnen in beide bestanden: {len(common_ids)}")

# aantal cellijnen die alleen in methylatie zitten
only_meth = meth_ids - mut_ids
print(f"Cellijnen alleen in methylatie: {len(only_meth)}")

# aantal cellijnen die alleen in mutaties zitten
only_mut = mut_ids - meth_ids
print(f"Cellijnen alleen in mutaties: {len(only_mut)}")



Aantal cellijnen na mapping: 153
                    locus_id  \
0  SGIP1_1_66998638_66999638   
2  AZIN2_1_33545713_33546713   
3  AZIN2_1_33546778_33547778   
4  AGBL4_1_50489626_50490626   
6  CLIC4_1_25070759_25071759   

                                      CpG_sites_hg19  avg_coverage  \
0  1:66998970;1:66998973;1:66998993;1:66999404;1:...         25.00   
2  1:33546151;1:33546209;1:33546210;1:33546385;1:...        326.58   
3  1:33546783;1:33546788;1:33546795;1:33546797;1:...        480.54   
4  1:50489632;1:50489641;1:50489671;1:50489677;1:...        263.36   
6  1:25070902;1:25070906;1:25071609;1:25071616;1:...        304.38   

   ACH-000698  ACH-000523  ACH-000610  ACH-000774  ACH-000875  ACH-000841  \
0     0.00000     0.11864     0.04545         NaN     0.25000     0.86441   
2     0.00729     0.22553     0.04560     0.14577     0.11487     0.17708   
3     0.22276     0.10735     0.03326     0.09589     0.21004     0.35571   
4     0.00000     0.07346     0.01246     0.0

#### Stap 3. Opschonen

Doel: ervoor zorgen dat alle bestanden dezelfde structuur en sleutel-IDs gebruiken.

De kolomnamen moeten worden gestandaardiseerd. Dit houd in dat alledrie de bestanden dezelfde sleutel-IDs moeten krijgen: cell_line_id. Ook moeten dubbele rijen of niet-unieke cellijnen moeten worden verwijderd. Er moet gecontroleerd worden op ontbrekende waarden (NaN) in de belangrijke kolommen. Dit is belangrijk om fouten te voorkomen bij het samenvoegen.

**xxx** avc_covverage in deze stap? > Waarden lager dan 10 worden verwijderd, omdat deze metingen onvoldoende dekking hebben om betrouwbare methylatiegegevens te geven. Dit voorkomt dat technische ruis invloed heeft op het model.

#### Stap 4. Transformatie van features

Doel: 
- mutatiegegevens omzetten naar binaire variabelen (0=mutatie, 1=mutatie).
- methylatiegegevens reduceren/aggregeren (gemiddelde pakken?)


Machine-learningmodellen werken met numerieke inputs; binaire encoding wordt veelgebruikt.

Methylatie heeft vaak duizenden features wat kan leiden tot overfitting? **meer uitleg nodig, bron**

Wanneer er meerdere AUC-metingen zijn per cellijn moet het gemiddelde worden berekend.

Aantal overlappende cellijnen: 0
Overlappende cellijnen: []


#### Stap 5. Samenvoegen 

Doel: het creëren van één tabel waarin elke rij één cellijn is, met alle relevante features en bijbehorende responswaarde.

De drie datasets worden samengevoegd op de gemeenschappelijke sleutel cell_line_id. 

Er wordt gebruik gemaakt van een **inner join** om ontbrekende waarden in de uiteindelijke tabel te voorkomen. Alleen cellijnen waarvan zowel moleculaire data (mutaties, methylatie) als responsdata beschikbaar zijn, worden behouden. Dit is een belangrijke eis voor supervised learning.

check ook hoeveel cellijnen verloren gaan door die joing.


#### Stap 6. Controle op ontbrekende waarden

Doel: controleren of de uiteindelijke dataset compleet genoeg is.

Cellijnen met onbrekende AUC moeten worden verwijderd omdat machine-learningalgoritmen niet kunnen omgaan met ontbrekende waarden. 



#### Stap 7. Preprocessing voor machine learning

Doel: de data omzetten naar numerieke vorm, schalen en voorbereiden voor modellering.

**hier start scikit-learn** Pandas is vooral geschikt voor het inlezen, bewerken en samenvoegen van dataframes. Scikit-learn is gespecialiseerd in preprocessing. 

**xxx** verwijs naar scikit-learn tutorial?




